In [7]:
from to_json import to_json
from typing import cast

from gdsfactory.pdk import get_layer_stack
from sky130 import LAYER_STACK

from kfactory import LayerEnum

from gdsfactory.component import Component
from gdsfactory.technology import DerivedLayer, LayerStack, LayerViews, LogicalLayer
from gdsfactory.typings import LayerSpecs
from gdsfactory.pdk import (
    get_active_pdk,
    get_layer,
    get_layer_stack,
    get_layer_views,
)
import gdsfactory as gf

2025-05-09 20:54:56.391 | INFO     | gdsfactory.pdk:register_cells_yaml:283 - Registered cell 'sample'


In [8]:
from polygon import group_by_length, group_congruent_polygons, plot_polygons, reconstruct_colored
from to_poly import to_poly

In [ ]:
c = gf.read.import_gds("example_sky130.gds")

layer_polygons = to_poly(c, layer_stack=LAYER_STACK)

In [ ]:
layer_polygons.keys()

In [ ]:
polygons = layer_polygons['polydrawing_m']

In [ ]:
groups = group_by_length(polygons)

In [ ]:
groups.keys()

In [ ]:
plot_polygons([(p, "rgba(48,48,48,1.0)") for p in polygons])

In [ ]:
groups2 = group_congruent_polygons(groups[8])

In [ ]:
groups2

In [ ]:
plot_polygons(reconstruct_colored(groups2))

In [ ]:
for g in groups.values():
    plot_polygons([(p, "rgba(48,48,48,1.0)") for p in g])
    groups2 = group_congruent_polygons(g)
    result = reconstruct_colored(groups2)
    plot_polygons(result)

# SCRATCH

In [ ]:
def to_poly(
    component: Component,
    layer_views: LayerViews | None = None,
    layer_stack: LayerStack | None = None,
    exclude_layers: LayerSpecs | None = None,
    return_json: bool = True,
):
    """Return optimzed json.

    Args:
        component: to extrude in 3D.
        layer_views: layer colors from Klayout Layer Properties file.
            Defaults to active PDK.layer_views.
        layer_stack: contains thickness and zmin for each layer.
            Defaults to active PDK.layer_stack.
        exclude_layers: list of layer index to exclude.

    """

    layer_views = layer_views or get_layer_views()
    layer_stack = layer_stack or get_layer_stack()

    exclude_layers = exclude_layers or ()
    exclude_layers = [get_layer(layer) for layer in exclude_layers]

    component_with_booleans = layer_stack.get_component_with_derived_layers(component)
    polygons_per_layer = component_with_booleans.get_polygons_points(merge=True)
    has_polygons = False

    top_name = "GDS"

    polygons = {}
    
    for level in layer_stack.layers.values():
        layer = level.layer

        if isinstance(layer, LogicalLayer):
            assert isinstance(layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(layer.layer))
        elif isinstance(layer, DerivedLayer):
            assert level.derived_layer is not None
            assert isinstance(level.derived_layer.layer, tuple | LayerEnum)
            layer_tuple = cast(tuple[int, int], tuple(level.derived_layer.layer))
        else:
            raise ValueError(f"Layer {layer!r} is not a DerivedLayer or LogicalLayer")

        layer_index = int(get_layer(layer_tuple))

        if layer_index in exclude_layers:
            continue

        if layer_index not in polygons_per_layer:
            continue

        zmin = level.zmin
        height = level.thickness
        layer_view = layer_views.get_from_tuple(layer_tuple)

        assert layer_view.fill_color is not None
        if zmin is not None and layer_view.visible:
            has_polygons = True
            polygons[layer_view.name] = polygons_per_layer[layer_index]

    if not has_polygons:
        raise ValueError(
            f"{component.name!r} does not have polygons defined in the "
            f"layer_stack or layer_views for the active Pdk {get_active_pdk().name!r}"
        )

    return polygons


In [ ]:
import numpy as np
import hashlib
from numba import njit
from collections import defaultdict
import plotly.graph_objects as go

from shapely.geometry import Polygon
from shapely import set_precision
from shapely import normalize

import distinctipy

TR = [
    lambda x,y: (+x, +y),  # Identity
    lambda x,y: (+y, -x),  # Rotate  90
    lambda x,y: (-x, -y),  # Rotate 180
    lambda x,y: (-y, +x),  # Rotate 270
    lambda x,y: (+x, -y),  # Mirror
    lambda x,y: (+y, +x),  # Mirror + Rotate  90
    lambda x,y: (-x, +y),  # Mirror + Rotate 180
    lambda x,y: (-y, -x),  # Mirror + Rotate 270
]

# ---------------------------
# 1. Core Congruence Detection
# ---------------------------

MAP = np.array([0, 1, 2, 3, 4, 5, 6, 7], dtype=np.int64)
INVERSE_MAP = np.array([0, 3, 2, 1, 4, 7, 6, 5], dtype=np.int64)

#@njit
def transformX(points, trans_index):
    transformed = np.empty_like(points)
    for i in range(len(points)):    
        transformed[i] = TR[trans_index](*points[i])
    return transformed
    
@njit
def transform(points, trans_idx):
    transformed = np.empty_like(points)

    for i in range(len(points)):
        x, y = points[i]
        if trans_idx == 0:    # Identity
            tx, ty = x, y
        elif trans_idx == 1:  # Rotate 90°
            tx, ty = y, -x
        elif trans_idx == 2:  # Rotate 180°
            tx, ty = -x, -y
        elif trans_idx == 3:  # Rotate 270°
            tx, ty = -y, x
        elif trans_idx == 4:  # Reflect over y-axis
            tx, ty = x, -y
        elif trans_idx == 5:  # Reflect + Rotate 90°
            tx, ty = y, x
        elif trans_idx == 6:  # Reflect + Rotate 180°
            tx, ty = -x, y
        elif trans_idx == 7:  # Reflect + Rotate 270°
            tx, ty = -y, -x
        transformed[i] = (tx, ty)
    return transformed

def center(points):
    centroid = (np.min(points, axis=0) + np.max(points, axis=0)) / 2
    return points - centroid, centroid

def group_by_length(polygons):
    groups = defaultdict(list)
    for polygon in polygons:
        groups[len(polygon)].append(polygon)
    return groups

def hash_polygon(polygon):
    return hash(normalize(set_precision(Polygon(polygon), grid_size=0.00001)))
    
def group_congruent_polygons(polygons):
    groups = {}
    
    for idx, poly in enumerate(polygons):
        centered_poly, centroid = center(poly)
        
        found = False
        for trans_index in MAP:    
            key = hash_polygon(transform(centered_poly, trans_index))
            if groups.get(key) is not None:        
                groups[key].append({
                    'index': idx,
                    'centroid': centroid,
                    'transformation': INVERSE_MAP[trans_index]
                })
                found = True
        
        if not found:
            # first element is the reference polygon
            key = hash_polygon(centered_poly)
            groups[key] = [centered_poly]
            
            # All other elements are instances
            groups[key].append({
                'index': idx,
                'centroid': centroid,
                'transformation': 0
            })
    return groups

# ---------------------------
# 2. Reconstruction Function
# ---------------------------

def reconstruct(groups, colored = False):
    result = []
    for i, (key, group) in enumerate(groups.items()):
        reference = group[0]
        for polygon in group[1:]:
            poly = transform(reference, INVERSE_MAP[polygon["transformation"]]) + polygon["centroid"]
            result.append(poly)
    return result

def reconstruct_colored(groups):
    result = []
    colors = [f"rgba({r},{g},{b},1.0)" for r,g,b in (np.array(distinctipy.get_colors(len(groups))) * 255).round(0)]
    for i, (key, group) in enumerate(groups.items()):
        reference = group[0]
        for polygon in group[1:]:
            poly = transform(reference, INVERSE_MAP[polygon["transformation"]]) + polygon["centroid"]
            result.append((poly, colors[i]))

    return result

# ---------------------------
# 3. Plot Polygons
# ---------------------------

def plot_polygons(polygons, title="Original Polygons", width=1300, height=1300):
    fig = go.Figure()
    
    for i, (poly, color) in enumerate(polygons):
        fill_color = color.replace("1.0", "0.3")
        closed = np.vstack((poly, poly[0]))
        fig.add_trace(go.Scatter(
            x=closed[:,0], y=closed[:,1],
            mode='lines',
            line=dict(color=color, width=1),
            fill='toself',
            fillcolor=fill_color,
            name=f'Polygon {i}'
        ))
    
    fig.update_layout(
        title=title,
        width=width, height=height,
        showlegend=False,
        plot_bgcolor='white',
        xaxis=dict(showgrid=False, zeroline=False, scaleanchor='y'),
        yaxis=dict(showgrid=False, zeroline=False, scaleanchor='x')
    )
    fig.show()

In [ ]:
c = gf.read.import_gds("example_sky130.gds")

layer_polygons = to_poly(c, layer_stack=LAYER_STACK)

In [ ]:
layer_polygons.keys()

In [ ]:
polygons = layer_polygons['polydrawing_m']

In [ ]:
groups = group_by_length(polygons)

In [ ]:
groups.keys()

In [ ]:
plot_polygons([(p, "rgba(48,48,48,1.0)") for p in polygons])

In [ ]:
for g in groups.values():
    plot_polygons([(p, "rgba(48,48,48,1.0)") for p in g])
    groups2 = group_congruent_polygons(g)
    result = reconstruct(groups2, True)
    plot_polygons(result)

In [ ]:
hash_polygon(center(g[411])[0]), hash_polygon(transform(center(g[428])[0],4))

In [ ]:

result = reconstruct(groups2, True)

In [ ]:
plot_polygons(result)

In [ ]:
groups2.keys()

In [ ]:
[f"rgba({r},{g},{b},1.0)" for r,g,b in (p.array(distinctipy.get_colors(len(groups))) * 255).round(0)]

In [ ]:
p = normalize(Polygon(polygons[1]))

In [ ]:
np.array(p.exterior.coords)

In [ ]:
# ---------------------------
# 3-4. Plotting Functions
# ---------------------------

def plot_polygons(polygons, title="Original Polygons", width=1300, height=1300):
    fig = go.Figure()
    color = "rgba(0.3, 0.3, 0.3, 1.0)"
    fill_color = "rgba(0.3, 0.3, 0.3, 0.2)"
    
    for i, poly in enumerate(polygons):
        closed = np.vstack((poly, poly[0]))
        fig.add_trace(go.Scatter(
            x=closed[:,0], y=closed[:,1],
            mode='lines',
            line=dict(color=color, width=1),
            fill='toself',
            fillcolor=fill_color
        ))
    
    fig.update_layout(
        title=title,
        width=width, height=height,
        showlegend=False,
        plot_bgcolor='white',
        xaxis=dict(showgrid=False, zeroline=False, scaleanchor='y'),
        yaxis=dict(showgrid=False, zeroline=False, scaleanchor='x')
    )
    fig.show()

def plot_congruence_groups(groups, width=900, height=900):
    for group_idx, group in enumerate(groups):
        fig = go.Figure()

        color = "rgba(0.3, 0.3, 0.3, 1.0)"
        fill_color = "rgba(0.3, 0.3, 0.3, 0.2)"
        
        # Reference polygon
        ref_closed = np.vstack((group['reference'], group['reference'][0]))
        fig.add_trace(go.Scatter(
            x=ref_closed[:,0], y=ref_closed[:,1],
            mode='lines',
            line=dict(color='black', width=1),
            marker=dict(size=8),
            fill='toself',
            fillcolor=fill_color
        ))
        
        # Reconstructed members
        for i, member in enumerate(group['members']):
            reconstructed = reconstruct_from_group(group, member)
            closed = np.vstack((reconstructed, reconstructed[0]))
            fig.add_trace(go.Scatter(
                x=closed[:,0], y=closed[:,1],
                mode='lines',
                line=dict(color=color, width=1),
                fill='toself',
                fillcolor=fill_color
            ))
        
        fig.update_layout(
            title=f'Group {group_idx} ({len(group["members"])} members)',
            width=width, height=height,
            showlegend=False,
            plot_bgcolor='white',
            xaxis=dict(showgrid=False, zeroline=False, scaleanchor='y'),
            yaxis=dict(showgrid=False, zeroline=False, scaleanchor='x')
        )
        fig.show()

In [ ]:
def plot(polygons, width=1300, height=1300):
    fig = go.Figure()
    rgb = ((0.3,0,0), (0,0.3,0), (0,0,0.3))
    
    for i, (poly) in enumerate(polygons):
        r,g,b = rgb[i]
        color = f"rgba({r}, {g}, {b}, 1.0)"
        fill_color = f"rgba({r}, {g}, {b}, 0.2)"
        closed = poly # np.vstack((poly, poly[0]))
        fig.add_trace(go.Scatter(
            x=closed[:,0], 
            y=closed[:,1],
            mode='lines',
            line=dict(color=color, width=1),
            fill='toself',
            fillcolor=fill_color,
            name=f'Polygon {i}'
        ))
    
        fig.update_layout(
            width=width,
            height=height,
            plot_bgcolor='white',
            paper_bgcolor='white',
            xaxis=dict(
                zeroline=True,
                zerolinecolor='lightgrey',
                showline=True,
                linecolor='lightgrey',
                mirror=True,
                anchor='y',
                position=0.5  # This centers the axis if using domain, see note below
            ),
            yaxis=dict(
                zeroline=True,
                zerolinecolor='lightgrey',
                showline=True,
                linecolor='lightgrey',
                mirror=True,
                anchor='x',
                position=0.5
            )
        )
    fig.show()

In [ ]:
p1 = list(groups2.values())[0][0]
p2 = list(groups2.values())[1][0]

In [ ]:
plot([p1, transform(p2,4)], 600, 600)

In [ ]:
from shapely.geometry import Polygon
from shapely import set_precision


ps1 = Polygon(p1)
ps2 = Polygon(transform(p2,4))
ps1 = set_precision(ps1, grid_size=0.00001)
ps2 = set_precision(ps2, grid_size=0.00001)
hash(normalize(ps1)), hash(normalize(ps2))

In [ ]:
ps1.equals(ps2)

In [ ]:
hash(normalize(ps1)), hash(normalize(ps2))

In [ ]:
from shapely.plotting import plot_polygon
import matplotlib.pyplot as plt

In [ ]:
plot([polygons[1155]], 600, 600)

In [ ]:
from shapely import normalize, to_wkb

ps = Polygon(polygons[1155])
set_precision(ps1, grid_size=0.00001)
print(to_wkb(normalize(ps)))

In [ ]:
print(ps)

In [ ]:
normalize?

In [ ]:
to_wkb

In [ ]:
# 1. Plot originals
plot_polygons(polygons)

In [ ]:
# 2. Group polygons
groups = group_congruent_polygons(polygons)

In [ ]:
len(groups)

In [ ]:
groups[0]["reference"]

In [ ]:
groups[1]["reference"]

In [ ]:
# 3. Plot groups
plot_congruence_groups(groups)

In [6]:
import numpy as np
from polygon import *

l = np.array([(0,0), (1,0), (1,-1), (-1,-1), (-1,2), (0,2)])

offsets = [(4,0), (8,0),(12,0), (16,0), (4,-4), (8,-4), (12,-4), (16,-4)]
polygons = [
    transform(l, t) + o for t, o in enumerate(offsets)
]

plot_polygons([(p, "rgba(48,48,48,1.0)") for p in polygons])

In [8]:
groups = group_congruent_polygons(polygons)
groups

{-1969481517017659905: [array([[ 0. , -0.5],
         [ 1. , -0.5],
         [ 1. , -1.5],
         [-1. , -1.5],
         [-1. ,  1.5],
         [ 0. ,  1.5]], dtype=float32),
  {'idx': 0, 'centroid': [4.0, 0.5], 'transformation': 0},
  {'idx': 1, 'centroid': [8.5, 0.0], 'transformation': 3},
  {'idx': 2, 'centroid': [12.0, -0.5], 'transformation': 2},
  {'idx': 3, 'centroid': [15.5, 0.0], 'transformation': 1},
  {'idx': 4, 'centroid': [4.0, -4.5], 'transformation': 4},
  {'idx': 5, 'centroid': [8.5, -4.0], 'transformation': 5},
  {'idx': 6, 'centroid': [12.0, -3.5], 'transformation': 6},
  {'idx': 7, 'centroid': [15.5, -4.0], 'transformation': 7}]}

In [ ]:
plot_polygons([(p, "rgba(48,48,48,1.0)") for p in reconstruct(groups)])

In [2]:
import orjson
from ocp_tessellate.utils import numpy_to_buffer_json

def polygons_to_json(polygon):

    top_name = "GDS"
    poly_assembly = {
        "format": "GDS",
        "version": 2,
        "name": top_name,
        "id": f"/{top_name}",
        "loc": [(0, 0, 0), (0, 0, 0, 1)],
        "instances": [],
        "parts": [],
    }
    ref = 0


    zmin = 0
    height = 1
    name = "Test"
    
    poly_parts = {
        "format": "GDS",
        "version": 2,
        "name": name,
        "id": f"/{top_name}/{name}",
        "loc": [(0, 0, zmin), (0, 0, 0, 1)],
        "parts": [],
    }

    index = 0
    groups_by_length = group_by_length(polygons).values()
    for groups in groups_by_length:
        congruent_polygons = group_congruent_polygons(groups)

        for group in congruent_polygons.values():
            poly_shape = {
                "name": f"group_{index}",
                "id": f"/{top_name}/{name}/group_{index}",
                "loc": [(0, 0, 0), (0, 0, 0, 1)],
                "color": "#ff0000",
                "shape": {
                    "ref": None,
                    "offsets": None,
                    "height": height,
                },
                "renderback": False,
                "state": [1, 1],
                "type": "polygon",
                "subtype": "solid",
            }
            index += 1

            poly_assembly["instances"].append(group[0])
            poly_shape["shape"]["ref"] = ref
            poly_shape["shape"]["offsets"] = np.asarray(
                [(*p["centroid"], p["transformation"]) for p in group[1:]],
                dtype="float32",
            )
            poly_parts["parts"].append(poly_shape)
            ref += 1

    poly_assembly["parts"].append(poly_parts)
    return poly_assembly
    return orjson.dumps(numpy_to_buffer_json(poly_assembly)).decode("utf-8")

In [4]:
name = "transformation_dummy"
with open(f"{name}.js", "w") as fd:
        j = polygons_to_json(polygons)
        fd.write(f"const {name} = {j};")

In [3]:
polygons_to_json(polygons)

{'format': 'GDS',
 'version': 2,
 'name': 'GDS',
 'id': '/GDS',
 'loc': [(0, 0, 0), (0, 0, 0, 1)],
 'instances': [array([[ 0. , -0.5],
         [ 1. , -0.5],
         [ 1. , -1.5],
         [-1. , -1.5],
         [-1. ,  1.5],
         [ 0. ,  1.5]], dtype=float32)],
 'parts': [{'format': 'GDS',
   'version': 2,
   'name': 'Test',
   'id': '/GDS/Test',
   'loc': [(0, 0, 0), (0, 0, 0, 1)],
   'parts': [{'name': 'group_0',
     'id': '/GDS/Test/group_0',
     'loc': [(0, 0, 0), (0, 0, 0, 1)],
     'color': '#ff0000',
     'shape': {'ref': 0,
      'offsets': array([[ 4. ,  0.5,  0. ],
             [ 8.5,  0. ,  3. ],
             [12. , -0.5,  2. ],
             [15.5,  0. ,  1. ],
             [ 4. , -4.5,  4. ],
             [ 8.5, -4. ,  5. ],
             [12. , -3.5,  6. ],
             [15.5, -4. ,  7. ]], dtype=float32),
      'height': 1},
     'renderback': False,
     'state': [1, 1],
     'type': 'polygon',
     'subtype': 'solid'}]}]}